# Grandes modelos de lenguaje (LLMs)

En este notebook, exploraremos el uso de grandes modelos de lenguaje (LLMs) para tareas de procesamiento de lenguaje natural, con hincapié en aplicaciones clínicas y de salud. Cubriremos cómo conectarse a APIs de LLMs, enviar consultas, procesar respuestas y algunos ejemplos de prompting y buenas prácticas.

In [1]:
import requests
from openai import OpenAI
from pydantic import BaseModel
import json
import datasets
from sklearn.metrics import classification_report

## Autenticación

Para interactuar con un LLM, primero debemos autenticar nuestra aplicación. Esto generalmente implica obtener una clave API de un proveedor de LLM. Asegúrate de tener tu clave API a mano.

In [ ]:
API_KEY = ""

## API de OpenAI

A través de consultas HTTP a la API de OpenAI, podemos enviar solicitudes y recibir respuestas de un modelo de lenguaje. Aquí hay un ejemplo básico de cómo hacerlo en Python.

In [3]:
url = "https://api.openai.com/v1/responses"
headers = {
    "Content-Type": "application/json",
}
headers["Authorization"] = f"Bearer {API_KEY}"
data = {
    "model": "gpt-4.1-nano",
    "input": "Define qué es el procesamiento de lenguaje natural clínico.",
}
response = requests.post(url, headers=headers, json=data)
response.json()

{'id': 'resp_6856312a0198819baa02d4b05a016e1a01f71055c5dbfaae',
 'object': 'response',
 'created_at': 1750479146,
 'status': 'completed',
 'background': False,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'model': 'gpt-4.1-nano-2025-04-14',
 'output': [{'id': 'msg_6856312ab190819bab2f8945b2464e4701f71055c5dbfaae',
   'type': 'message',
   'status': 'completed',
   'content': [{'type': 'output_text',
     'annotations': [],
     'text': 'El procesamiento de lenguaje natural clínico (PLNC) es una rama especializada del procesamiento de lenguaje natural (PLN) que se enfoca en la interpretación, análisis y extracción de información de textos y registros clínicos o médicos. Este campo combina técnicas de inteligencia artificial, aprendizaje automático y lingüística computacional para facilitar la gestión, el análisis y la utilización de datos clínicos escritos en lenguaje natural, como notas médicas, informes de laboratorios, historial clí

## Uso de la biblioteca `openai`

En vez de hacer solicitudes HTTP manualmente, podemos usar la biblioteca `openai` para simplificar el proceso. Asegúrate de instalarla primero:


In [4]:
client = OpenAI(
    api_key=API_KEY
)

###  Uso básico

Podemos usar la biblioteca `openai` para enviar solicitudes a la API de OpenAI. Aquí hay un ejemplo básico de cómo hacerlo

In [5]:
response = client.responses.create(
    model="gpt-4o",
    instructions="Eres un experto en procesamiento de lenguaje natural clínico. Responde a la pregunta de manera clara y concisa.",
    input="¿Qué debo tomar en cuenta para desarrollar una función de preprocesamiento?",
)

print(response.output_text)

Para desarrollar una función de preprocesamiento en el contexto del procesamiento de lenguaje natural clínico, es importante considerar lo siguiente:

1. **Estandarización de términos**: Unifica terminologías médicas utilizando ontologías como SNOMED CT o UMLS para obtener consistencia.

2. **Eliminación de ruido**: Remueve información irrelevante, como números de expediente o datos de identificación personal, para proteger la privacidad.

3. **Tokenización**: Separa el texto en unidades significativas (tokens), tomando en cuenta las características del lenguaje médico.

4. **Normalización**: Convierte abreviaturas y diferentes formatos de términos a una forma estándar.

5. **Corrección ortográfica**: Aborda errores tipográficos comunes en textos médicos.

6. **Identificación de entidades**: Detecta y marca entidades médicas relevantes, como enfermedades, medicamentos, y procedimientos.

7. **Desambiguación**: Resuelve ambigüedades contextuales en términos médicos.

8. **Filtrado de st

También podemos usar imágenes como parte de nuestras consultas. A continuación, se muestra un ejemplo de cómo enviar una imagen junto con un mensaje de texto.


In [6]:
prompt = "¿Qué enfermedad es probable que tenga el paciente?"
img_url = "https://patoral.umayor.cl/canmucor/ca_leng_mb1.jpg"

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"{img_url}"},
            ],
        }
    ],
)

print(response.output_text)

La imagen muestra lesiones en la cavidad oral que podrían estar asociadas con varias afecciones. Una posibilidad común podría ser la leucoplasia oral, que aparece como manchas blancas o placas en las mucosas. También podría ser un signo de candidiasis oral o incluso una enfermedad periodontal avanzada. Para un diagnóstico adecuado, es importante que el paciente consulte a un profesional de salud.


### Mensajes y roles

Los mensajes en la API de OpenAI tienen roles que indican quién está hablando. Los roles comunes son `system`, `user` y `assistant`. Aquí hay un ejemplo de cómo estructurar un mensaje

In [7]:
response = client.responses.create(
    model="gpt-4.1",
    input=[
        {"role": "developer", "content": "Eres un experto en salud digital."},
        {"role": "user", "content": "Qué es lo más importante al implementar un proyecto de informática médica?"}
    ]
)
print(response.output_text)


¡Excelente pregunta! Como experto en salud digital, te puedo decir que la implementación exitosa de un proyecto de informática médica (como una historia clínica electrónica, un sistema de laboratorio o de gestión hospitalaria) depende de varios factores críticos. Los más importantes suelen ser:

**1. Enfoque centrado en el usuario:**  
Antes que la tecnología misma, lo primordial es que el sistema satisfaga las necesidades clínicas y administrativas de quien lo usará: médicos, enfermería, personal administrativo, pacientes, etc. Involucra a los futuros usuarios desde el principio, escúchalos, recaba feedback y haz pruebas piloto.

**2. Gestión del cambio:**  
Los proyectos de informática médica no son solo de tecnología, sino de transformación cultural. La resistencia al cambio es uno de los principales motivos de fracaso. Planifica capacitación, comunicación y acompañamiento durante toda la implementación.

**3. Interoperabilidad y estándares:**  
El sistema debe poder intercambiar da

In [8]:
developer_message = """
# Identidad

Eres un priorizador de la lista de espera chilena y deben priorizar pacientes en las categorías Urgente o Rutina.

# Instrucciones

* Solo responde con una palabra: "Urgente" o "Rutina".
"""

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": "El paciente presenta un dolor muy leve en el dedo meñique izquierdo."}
    ]
)
print(response.output_text)


Rutina


### Adición de ejemplos

Para mejorar la calidad de las respuestas, podemos proporcionar ejemplos de preguntas y respuestas esperadas. Esto ayuda al modelo a entender mejor el contexto y las expectativas.

In [9]:
developer_message = """
# Identidad

Eres un priorizador de la lista de espera chilena y deben priorizar pacientes en las categorías Urgente o Rutina.

# Instrucciones

* Solo responde con una palabra: "Urgente" o "Rutina".

# Ejemplos

<interconsulta>
El paciente presenta pérdida de peso de 10 kg en los últimos 3 meses y tiene antecedentes de cáncer de colon.
</interconsulta>
<assistant_response">
Urgente
</assistant_response>
<interconsulta>
El paciente presenta un dolor muy leve en el dedo meñique izquierdo.
</interconsulta>
<assistant_response">
Rutina
</assistant_response>
<interconsulta>
El paciente tiene un dolor intenso en el pecho y dificultad para respirar.
</interconsulta>
<assistant_response">
Urgente
</assistant_response>
"""

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": "El paciente tiene un dolor intenso en la zona parietal izquierda."}
    ]
)
print(response.output[0].content[0].text)


Urgente


### Respuesta estructurada

Para obtener respuestas más estructuradas, podemos usar el parámetro `text_format` en la solicitud. Esto nos permite especificar el formato de la respuesta esperada.

In [10]:
# Definir el esquema de salida estructurada
class InterconsultaPrioridad(BaseModel):
    prioridad: str  # "Urgente" o "Rutina"

# Usar el método parse para obtener una respuesta estructurada
response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Eres un priorizador de la lista de espera chilena. Responde solo con la prioridad del paciente ('Urgente' o 'Rutina')."},
        {"role": "user", "content": "El paciente tiene fiebre alta y dificultad respiratoria."}
    ],
    text_format=InterconsultaPrioridad,
)

print(response.output_parsed.prioridad)

Urgente


En algunos casos, es fundamental que la respuesta del modelo siga una estructura específica y validable, especialmente para integraciones clínicas o flujos automatizados. La API de OpenAI permite definir un esquema JSON (JSON Schema) que el modelo debe seguir estrictamente en su respuesta. A continuación, se muestra cómo definir un esquema para priorización de interconsultas y cómo solicitar al modelo que responda usando exactamente ese formato estructurado.

In [11]:
schema = {
    "type": "object",
    "properties": {
        "prioridad": {
            "type": "string",
            "enum": ["Urgente", "Rutina"],
            "description": "Prioridad del paciente según la interconsulta"
        },
        "descripcion": {
            "type": "string",
            "description": "Descripción adicional de la interconsulta"
        }
    },
    "required": ["prioridad", "descripcion"],
    "additionalProperties": False
}

response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Eres un priorizador de la lista de espera chilena. Responde solo con la prioridad del paciente ('Urgente' o 'Rutina')."},
        {"role": "user", "content": "El paciente refiere dolor abdominal leve desde hace 2 semanas."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "prioridad_interconsulta",
            "schema": schema,
            "strict": True
        }
    }
)

json.loads(response.output[0].content[0].text)

{'prioridad': 'Rutina',
 'descripcion': 'El dolor abdominal leve presente desde hace 2 semanas sugiere una condición que no requiere atención inmediata, pero debe ser evaluada en un próximo control médico de rutina.'}

### Chain of Thought (Cadena de Pensamiento)

El prompting tipo "chain of thought" (cadena de pensamiento) le indica al modelo que razone paso a paso antes de dar una respuesta final. Esto es útil para tareas complejas donde se requiere justificar o explicar el razonamiento detrás de la decisión.

A continuación, se muestra un ejemplo donde se le pide al modelo que explique su razonamiento antes de priorizar la interconsulta


In [12]:
cot_prompt = """
Eres un priorizador de la lista de espera chilena. 
Primero, analiza los síntomas del paciente paso a paso y explica tu razonamiento. 
Luego, responde con la prioridad final: "Urgente" o "Rutina".

Paciente: El paciente presenta fiebre alta, tos persistente y dificultad respiratoria.
"""

response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": cot_prompt}
    ]
)

print(response.output_text)

Análisis de los síntomas del paciente:

1. **Fiebre alta**:
   - La fiebre alta puede indicar una infección o inflamación en el cuerpo. Es un síntoma general que, combinado con otros, puede señalar una condición más seria.

2. **Tos persistente**:
   - Una tos que no cede puede ser un signo de una infección respiratoria, como bronquitis o neumonía. Es particularmente preocupante si interfiere con la respiración.

3. **Dificultad respiratoria**:
   - Este es un síntoma significativo que indica que el paciente tiene problemas para obtener suficiente oxígeno. Puede ser una señal de una afección respiratoria grave, como neumonía, enfermedad pulmonar obstructiva crónica (EPOC), o incluso COVID-19.

**Razón para la priorización**:
La combinación de fiebre alta, tos persistente y dificultad respiratoria sugiere una posible infección respiratoria aguda. La dificultad respiratoria es especialmente preocupante ya que puede comprometer la oxigenación y requiere evaluación médica inmediata.

**Pri

## Clasificador utilizando LLMs

Los LLMs también pueden ser utilizados como clasificadores para tareas específicas, como la clasificación de interconsultas médicas. A continuación, se muestra un ejemplo de cómo implementar un clasificador utilizando un LLM.

In [13]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics')

In [14]:
test = spanish_diagnostics['test'].select(range(100))

In [15]:
def classifier(text):
    schema = {
        "type": "object",
        "properties": {
            "tipo": {
                "type": "string",
                "enum": ["dental", "no_dental"],
                "description": "Tipo de diagnóstico: 'dental' cuando se debe enviar la interconsulta a una especialidad dental o 'no_dental' cuando no se debe enviar a una especialidad dental"
            }
        },
        "required": ["tipo"],
        "additionalProperties": False
    }
    response = client.responses.create(
        model="gpt-4o",
        input=[
            {"role": "system", "content": "Eres un clasificador de diagnósticos médicos."},
            {"role": "user", "content": text}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "diagnostico_clasificacion",
                "schema": schema,
                "strict": True
            }
        }
    )
    return json.loads(response.output[0].content[0].text)["tipo"]

In [16]:
classifier("caries en el diente 12 y 13, dolor leve al masticar")

'dental'

In [17]:
predicted = [classifier(item['text']) for item in test]

In [18]:
print(classification_report(["dental" if item['label'] == 1 else "no_dental" for item in test], predicted))

              precision    recall  f1-score   support

      dental       0.93      0.98      0.96        44
   no_dental       0.98      0.95      0.96        56

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100



In [19]:
def classifier_few_shot(text):
    schema = {
        "type": "object",
        "properties": {
            "tipo": {
                "type": "string",
                "enum": ["dental", "no_dental"],
                "description": "Tipo de diagnóstico: 'dental' cuando se debe enviar la interconsulta a una especialidad dental o 'no_dental' cuando no se debe enviar a una especialidad dental"
            }
        },
        "required": ["tipo"],
        "additionalProperties": False
    }
    response = client.responses.create(
        model="gpt-4o",
        input=[
            {"role": "system", "content": "Eres un clasificador de diagnósticos médicos."},
            {"role": "user", "content": "paciente con dolor en el diente 12 y 13"},
            {"role": "assistant", "content": '{"tipo":"dental"}'},
            {"role": "user", "content": "paciente con dolor en la rodilla derecha"},
            {"role": "assistant", "content": '{"tipo":"no_dental"}'},
            {"role": "user", "content": text}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "diagnostico_clasificacion",
                "schema": schema,
                "strict": True
            }
        }
    )
    return json.loads(response.output[0].content[0].text)["tipo"]

In [20]:
classifier_few_shot("El paciente presenta dolor en el diente 12 y 13, con sensibilidad al frío y al calor.")

'dental'

In [21]:
predicted = [classifier_few_shot(item['text']) for item in test]

In [22]:
print(classification_report(["dental" if item['label'] == 1 else "no_dental" for item in test], predicted))

              precision    recall  f1-score   support

      dental       0.94      1.00      0.97        44
   no_dental       1.00      0.95      0.97        56

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100

